# Biostat 257 Homework 2

**Due Apr 30 @ 11:59PM**

In [ ]:
versioninfo()

Consider a linear mixed effects model
$$
    \mathbf{Y}_i = \mathbf{X}_i \boldsymbol{\beta} + \mathbf{Z}_i \boldsymbol{\gamma} + \boldsymbol{\epsilon}_i, \quad i=1,\ldots,n,
$$
where   
- $\mathbf{Y}_i \in \mathbb{R}^{n_i}$ is the response vector of $i$-th individual,  
- $\mathbf{X}_i \in \mathbb{R}^{n_i \times p}$ is the fixed effect predictor matrix of $i$-th individual,  
- $\mathbf{Z}_i \in \mathbb{R}^{n_i \times q}$ is the random effect predictor matrix of $i$-th individual,  
- $\boldsymbol{\epsilon}_i \in \mathbb{R}^{n_i}$ are multivariate normal $N(\mathbf{0}_{n_i},\sigma^2 \mathbf{I}_{n_i})$,  
- $\boldsymbol{\beta} \in \mathbb{R}^p$ are fixed effects, and  
- $\boldsymbol{\gamma} \in \mathbb{R}^q$ are random effects assumed to be $N(\mathbf{0}_q, \boldsymbol{\Sigma}_{q \times q}$) independent of $\boldsymbol{\epsilon}_i$.

## Q1 Formula (10 pts)

Write down the log-likelihood of the $i$-th datum $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$ given parameters $(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2)$. 

**Hint:** For non-statisticians, feel free to ask for help in class or office hours. Point of this exercise is computing not statistics.

$$Var(Y) = \mathbf{Z_i \Sigma Z_i^T} + \sigma^2 \mathbf{I}$$
$$l(\beta, \Sigma, \sigma^2) = -\frac{1}{2}\ln|2\pi(\mathbf{Z_i \Sigma Z_i^T} + \sigma^2 \mathbf{I_{n_i}})| - \frac{1}{2}(\mathbf{y_i} - \mathbf{X_i\beta})^T[\mathbf{Z_i \Sigma Z_i^T} + \sigma^2 \mathbf{I_{n_i}}]^{-1}(\mathbf{y_i} - \mathbf{X_i\beta})$$

We haven't covered log-likelihood functions in 200C / for mixed linear models, so I didn't provide a derivation.

## Q2 Start-up code

Use the following template to define a type `LmmObs` that holds an LMM datum $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$. 

Woodbury Identity:

$$[\mathbf{Z_i \Sigma Z_i^T} + \sigma^2 \mathbf{I_{n_i}}]^{-1} $$
$$= [\mathbf{(Z_i L) (Z_i L)^T} + \sigma^2 \mathbf{I_{n_i}}]^{-1}$$
$$ = [\sigma^2 \mathbf{I_{n_i}}]^{-1} - [\sigma^2 \mathbf{I_{n_i}}]^{-1}\mathbf{Z_iL}[I + (\mathbf{(Z_iL)^T}(\sigma^2 \mathbf{I_{n_i}})^{-1}\mathbf{(Z_iL)^T})^{-1}]\mathbf{(Z_iL)^T}[\sigma^2 \mathbf{I_{n_i}}]^{-1}$$
$$ = \frac{1}{\sigma^2}\mathbf{I} -\frac{1}{\sigma^4}\mathbf{Z_iL}[\mathbf{I} + \frac{1}{\sigma^2}(\mathbf{Z_iL})^T(\mathbf{Z_iL})]^{-1}(\mathbf{Z_iL})^T $$

Our term $(\mathbf{Z_iL})^T(\mathbf{Z_iL})$ is symmetric and positive definite. Adding it to the identity matrix maintains this property. Thus, Cholesky decomposition may be applied:

$$ \mathbf{I} + \frac{1}{\sigma^2}(\mathbf{Z_iL})^T(\mathbf{Z_iL}) = \mathbf{AA^T} $$
$$ [\mathbf{Z_i \Sigma Z_i^T} + \sigma^2 \mathbf{I_{n_i}}]^{-1} = \frac{1}{\sigma^2}\mathbf{I} - \frac{1}{\sigma^4}\mathbf{Z_iL}(\mathbf{AA^T})^{-1}(\mathbf{Z_iL})^T $$
$$ =\frac{1}{\sigma^2}\mathbf{I} - \frac{1}{\sigma^4}\mathbf{Z_iL}\mathbf{A^{-T}}\mathbf{A^{-1}}(\mathbf{Z_iL})^T $$
$$ = \frac{1}{\sigma^2}\mathbf{I} - \frac{1}{\sigma^4}(\mathbf{A^{-1}(Z_iL)^T})^T(\mathbf{A^{-1}(Z_iL)^T})$$

Computing the determinant term using $\text{det}(\mathbf{A} + \mathbf{U} \mathbf{V}^T) = \text{det}(\mathbf{A}) \text{det}(\mathbf{I}_m + \mathbf{V}^T \mathbf{A}^{-1} \mathbf{U})$:

$$-\frac{1}{2}\ln|2\pi(\mathbf{Z_i \Sigma Z_i^T} + \sigma^2 \mathbf{I_{n_i}})| = -\frac{1}{2}\ln(2\pi^{n_i}) -\frac{1}{2}\ln[|\mathbf{(Z_i L) (Z_i L)^T} + \sigma^2 \mathbf{I_{n_i}}|]$$
$$ = -\frac{1}{2}\ln(2\pi^n) - \frac{1}{2}\ln[|\sigma^2\mathbf{I_{n_i}}||\mathbf{I} + \frac{1}{\sigma^2}(\mathbf{Z_iL})^T(\mathbf{Z_iL})|]$$
$$ = -\frac{n}{2}\ln(2\pi) - \frac{1}{2}\ln((\sigma^2)^n) - \frac{1}{2}\ln|\mathbf{AA^T}| $$
$$ = -\frac{n}{2}\ln(2\pi) - \frac{n}{2}\ln(\sigma^2) - \frac{1}{2}\ln|\mathbf{AA^T}| $$

Final simplification:

$$ l(\beta, \Sigma, \sigma^2) = -\frac{n}{2}\ln(2\pi) - \frac{n}{2}\ln(\sigma^2) - \frac{1}{2}\ln|\mathbf{AA^T}| - \frac{1}{2}(\mathbf{y_i} - \mathbf{X_i\beta})^T[\frac{1}{\sigma^2}\mathbf{I} - \frac{1}{\sigma^4}(\mathbf{A^{-1}(Z_iL)^T})^T(\mathbf{A^{-1}(Z_iL)^T})](\mathbf{y_i} - \mathbf{X_i\beta})$$
$$ = -\frac{n}{2}\ln(2\pi) - \frac{n}{2}\ln(\sigma^2) - \frac{1}{2}\ln|\mathbf{AA^T}| - \frac{1}{2\sigma^2}(\mathbf{y_i} - \mathbf{X_i\beta})^T(\mathbf{y_i} - \mathbf{X_i\beta}) + \frac{1}{2\sigma^4}(\mathbf{y_i} - \mathbf{X_i\beta})^T(\mathbf{A^{-1}(Z_iL)^T})^T(\mathbf{A^{-1}(Z_iL)^T})(\mathbf{y_i} - \mathbf{X_i\beta})$$
$$ = -\frac{n}{2}\ln(2\pi) - \frac{n}{2}\ln(\sigma^2) - \frac{1}{2}\ln|\mathbf{AA^T}| - \frac{1}{2\sigma^2}(\mathbf{y_i} - \mathbf{X_i\beta})^T(\mathbf{y_i} - \mathbf{X_i\beta}) + \frac{1}{2\sigma^4}[(\mathbf{A^{-1}(Z_iL)^T})(\mathbf{y_i} - \mathbf{X_i\beta})]^T[(\mathbf{A^{-1}(Z_iL)^T})(\mathbf{y_i} - \mathbf{X_i\beta})]$$
$$ = -\frac{n}{2}\ln(2\pi) - \frac{n}{2}\ln(\sigma^2) - \frac{1}{2}\ln|\mathbf{AA^T}| - \frac{1}{2\sigma^2}(\mathbf{y^Ty} - 2\mathbf{y^TX\beta} + \mathbf{\beta^TX^TX\beta)} + \frac{1}{2\sigma^4}[(\mathbf{A^{-1}L^T})(\mathbf{Z^TY - Z^TX\beta)}]^T[(\mathbf{A^{-1}L^T})(\mathbf{Z^TY - Z^TX\beta)}]$$

Write a function, with interface   
```julia
logl!(obs, β, L, σ²)
```
that evaluates the log-likelihood of the $i$-th datum. Here `L` is the lower triangular Cholesky factor from the Cholesky decomposition `Σ=LL'`. Make your code efficient in the $n_i \gg q$ case. Think the intensive longitudinal measurement setting.  

In [1]:
struct LmmObs{T <: AbstractFloat}
    # data
    y :: Vector{T}
    X :: Matrix{T}
    Z :: Matrix{T}
    # working arrays
    # whatever intermediate arrays you may want to pre-allocate
    res :: Vector{T}
    det :: Matrix{T}
    det1 :: Matrix{T}
    det2 :: Matrix{T} 
    ztz :: Matrix{T}
    yty :: T
    xty :: Vector{T}
    xtx :: Matrix{T}
    ztx :: Matrix{T}
    zty :: Vector{T}
    last :: Vector{T}
end

# constructor
function LmmObs(
        y::Vector{T}, 
        X::Matrix{T}, 
        Z::Matrix{T}) where T <: AbstractFloat
    res = similar(y)
    ztz = transpose(Z) * Z
    det = similar(ztz)
    det1 = similar(ztz)
    det2 = similar(ztz)
    yty = transpose(y) * y
    xty = transpose(X) * y
    xtx = transpose(X) * X
    ztx = transpose(Z) * X
    zty = transpose(Z) * y
    last = Vector{T}(undef, size(Z, 2))
    
    LmmObs(y, X, Z, res, det, det1, det2, ztz, yty, xty, xtx, ztx, zty, last)
end

LmmObs

In [2]:
using LinearAlgebra
function logl!(
        obs :: LmmObs{T}, 
        β   :: Vector{T}, 
        L   :: Matrix{T}, 
        σ²  :: T) where T <: AbstractFloat
    n, p, q = size(obs.X, 1), size(obs.X, 2), size(obs.Z, 2) 
    
    #compute determinant term 
    mul!(obs.det, L', obs.ztz)
    mul!(obs.det1, obs.det, L)
    mul!(obs.det2, obs.det1, 1/σ²)
    
    iden = 1 * Matrix(I, size(obs.det2, 1), size(obs.det2, 1))
    axpy!(1, iden, obs.det2)
    
    chol = cholesky!(Symmetric(obs.det2)) 
        
    #compute last term
    mul!(obs.last, obs.ztx, β)
    axpby!(1, obs.zty, -1, obs.last)
    
    return -n//2 * log(2π) - n//2 * log(σ²) - 1//2 * logdet(chol) - 
        1 / (2 * σ²) * (obs.yty - 2 * obs.xty' * β + β' * obs.xtx * β)  + 
        1 / (2 * σ²^2) * dot(L' * obs.last, chol \ L' * obs.last)
end

logl! (generic function with 1 method)

**Hint**: This function shouldn't be very long. Mine, obeying 80-character rule, is 25 lines. If you find yourself writing very long code, you're on the wrong track. Think about algorithm first then use BLAS functions to reduce memory allocations.

## Q3 Correctness (15 pts)

Compare your result (both accuracy and timing) to the [Distributions.jl](https://juliastats.org/Distributions.jl/stable/multivariate/#Distributions.AbstractMvNormal) package using following data.

In [3]:
using BenchmarkTools, Distributions, LinearAlgebra, Random

Random.seed!(257)
# dimension
n, p, q = 2000, 5, 3
# predictors
X  = [ones(n) randn(n, p - 1)]
Z  = [ones(n) randn(n, q - 1)]
# parameter values
β  = [2.0; -1.0; rand(p - 2)]
σ² = 1.5
Σ  = fill(0.1, q, q) + 0.9I
# generate y
y  = X * β + Z * rand(MvNormal(Σ)) + sqrt(σ²) * randn(n)

# form an LmmObs object
obs = LmmObs(y, X, Z)

LmmObs{Float64}([5.982282543893974, 2.1686591674794546, -0.600617976847127, -2.5921722427552263, -4.1950904215114395, 0.19644359912941595, -1.8584206360155058, -0.471394614388984, 3.8132743617962293, 2.70502980503809  …  -0.9698126660920986, -0.056282731487944146, 0.1361331008072202, -0.27281934708288147, 3.204079310642381, 3.1301513446894242, -0.7337423498389697, -1.1992972996496185, 1.552593065317924, 0.9964276305957047], [1.0 -2.506566300781151 … -0.18291088048140966 0.4598620195142903; 1.0 -0.974090320735282 … -0.7383659530381397 0.4874285643091131; … ; 1.0 0.3528183431516365 … 1.9292747611484156 1.2067313494938754; 1.0 0.019416493632924994 … -1.3213407131023014 -0.06848691909471624], [1.0 0.8585392934004743 -1.9174716700008398; 1.0 0.9654277213047047 0.40862623314285934; … ; 1.0 -0.2387107330196111 -2.0711110232845926; 1.0 -0.9537172982914403 -1.1799476703506613], [NaN, 9.20483112e-315, NaN, 8.487983165e-314, 9.20483428e-315, 9.20482798e-315, NaN, NaN, 9.18964386e-315, 9.870701296

This is the standard way to evaluate log-density of a multivariate normal, using the Distributions.jl package. Let's evaluate the log-likelihood of this datum.

In [4]:
μ  = X * β
Ω  = Z * Σ * transpose(Z) +  σ² * I
mvn = MvNormal(μ, Symmetric(Ω)) # MVN(μ, Σ)
logpdf(mvn, y)

-3261.9177559187597

Check that your answer matches that from Distributions.jl

In [5]:
L = Matrix(cholesky(Σ).L)
logl!(obs, β, L, σ²)

-3261.91775591876

**You will lose all 15 + 30 + 30 = 75 points** if the following statement throws `AssertionError`.

In [6]:
@assert logl!(obs, β, Matrix(cholesky(Σ).L), σ²) ≈ logpdf(mvn, y)

## Q4 Efficiency (30 pts)

Benchmarking your code and compare to the Distributions.jl function `logpdf`.

In [7]:
# benchmark the `logpdf` function in Distribution.jl
bm1 = @benchmark logpdf($mvn, $y)

BenchmarkTools.Trial: 
  memory estimate:  30.55 MiB
  allocs estimate:  5
  --------------
  minimum time:     14.158 ms (0.00% GC)
  median time:      16.510 ms (0.00% GC)
  mean time:        18.138 ms (13.86% GC)
  maximum time:     28.277 ms (33.95% GC)
  --------------
  samples:          276
  evals/sample:     1

In [8]:
# benchmark your implementation
L = Matrix(cholesky(Σ).L)
bm2 = @benchmark logl!($obs, $β, $L, $σ²)

BenchmarkTools.Trial: 
  memory estimate:  944 bytes
  allocs estimate:  9
  --------------
  minimum time:     1.180 μs (0.00% GC)
  median time:      1.510 μs (0.00% GC)
  mean time:        1.543 μs (2.27% GC)
  maximum time:     353.940 μs (98.79% GC)
  --------------
  samples:          10000
  evals/sample:     10

The points you will get is
$$
\frac{x}{10000} \times 30,
$$
where $x$ is the speedup of your program against the standard method.

In [9]:
# this is the points you'll get
clamp(median(bm1).time / median(bm2).time / 10000 * 30, 0, 30)

30.0

**Hint**: Apparently I am using 10000 as denominator because I expect your code to be at least $10000 \times$ faster than the standard method.

## Q5 Memory (30 pts)

You want to avoid memory allocation in the "hot" function `logl!`. You will lose 1 point for each `1 KiB = 1024 bytes` memory allocation. In other words, the points you get for this question is

In [10]:
clamp(30 - median(bm2).memory / 1024, 0, 30)

29.078125

**Hint**: I am able to reduce the memory allocation to 0 bytes.

## Q6 Misc (15 pts)

Coding style, Git workflow, etc. For reproducibity, make sure we (TA and myself) can run your Jupyter Notebook. That is how we grade Q4 and Q5. If we cannot run it, you will get zero points.